In [ ]:
from pathlib import Path

import polars as pl
import polars.selectors as cs

### Read CSV

In [ ]:
movie_metadata_path = Path.cwd() / "movie_metadata.csv"

# mm = movie_metadata dataframe.
mm = pl.read_csv(source=movie_metadata_path)

In [ ]:
mm.head()

### Explore the Data
#### Number of Columns Per Data Type

In [ ]:
pl.Series(mm.dtypes).value_counts().sort(by="count", descending=True)

#### Column Names Per Data Type
##### Integer Columns

In [ ]:
mm.select(cs.integer()).columns

##### String Columns

In [ ]:
mm.select(cs.string()).columns

##### Float Columns

In [ ]:
mm.select(cs.float()).columns